# 生成2020训练数据

In [ ]:
from pyhive import hive
data_conn = hive.Connection(host='10.254.5.100',
                           port=10000,
                           username='ETL',
                           database='default')
cur = data_conn.cursor()


import pandas as pd
# open('../sql/train_2020.sql','r').read()
data = pd.read_sql(open('../sql/train_2020.sql','r').read(),data_conn)

import pickle as pl
pl.dump(data,open('train_data_2020','wb'))

In [ ]:
data

# #划分站区

In [1]:
from pyhive import hive
import pandas as pd
data_conn = hive.Connection(host='10.254.5.100',
                           port=10000,
                           username='ETL',
                           database='default')
full = pd.read_sql('select entry_station_id as station_id, stat_dt, entry_list from bmnc_infer.train_entry_od_data_re t where stat_dt between "2018-01-01" and  "2018-12-31" and (length(entry_station_id) == 4 and entry_station_id not in ("83","84","85","93","04"))',data_conn)

In [2]:
sql_rela = """
    select 
    stat_station_id as station_id,
    xfer_station_id as pId
    from 
     bmnc_pcodevw.t99_stat_stn_xfer_rela_his t
    where t.stat_std_cd = '10'
    and t.start_tm <= '2018-12-14'
    and t.end_tm >= '2018-12-14'
    and (length(t.stat_station_id) == 4 and t.stat_station_id not in ('83','84','85','93','04'))
    """

station_arr_sql = """
        select stat_station_id as station_id,stat_line_id as line_id from bmnc_pcode.t99_stat_station_cd
        where stat_line_id not in ('82' '83' '84' '85' '93' '04') 
        and open_tm <= '2018-12-14' and close_tm > '2018-12-14'
        order by station_id
        """

In [3]:
import numpy as np
phy_log_rela = pd.read_sql(sql_rela,data_conn)


ls = pd.read_sql(station_arr_sql,data_conn)


station_arr  = np.array(ls['station_id'].values)

In [4]:
full.head()

,station_id,stat_dt,entry_list
0,1427,2018-01-07,"[[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,..."
1,1427,2018-01-18,"[[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,..."
2,1427,2018-01-19,"[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,..."
3,1427,2018-01-21,"[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,..."
4,1427,2018-01-27,"[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,..."


In [5]:
phy_log_rela.head()

,station_id,pid
0,0110,H22
1,0111,H21
2,0114,H07
3,0115,H06
4,0119,H05


In [6]:
import numpy as np
full['line_id'] = full['station_id'].apply(lambda x: x[:2] if len(x) ==4 else x[:4])
full['year'] = full['stat_dt'].apply(lambda x: x[:4])
all = pd.merge(full,phy_log_rela,how='left',on = 'station_id')

# full['entry_list_'] = full['entry_list'].apply(lambda x: np.sum(np.array(eval(x)),axix = 0))

In [7]:
all.head()

,station_id,stat_dt,entry_list,line_id,year,pid
0,1427,2018-01-07,"[[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...",14,2018,NaN
1,1427,2018-01-18,"[[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,...",14,2018,NaN
2,1427,2018-01-19,"[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,...",14,2018,NaN
3,1427,2018-01-21,"[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...",14,2018,NaN
4,1427,2018-01-27,"[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...",14,2018,NaN


In [ ]:
station = station_arr[0]
one_all = all[all['station_id']==station]


In [ ]:
one_all['entry_day'] = one_all['entry_list'].apply(lambda x: np.sum(np.array(eval(x)),axis=0))

In [ ]:
line_id = one_all.iloc[0,0]
line_id

In [ ]:
# one_all.head()
# one_all['avg'] = [i for i in np.mean(one_all['entry_day'].values,axis=0)]
year_avg = np.mean(one_all['entry_day'].values,axis=0)
res = pd.DataFrame(
    {'station_id':[station],
        'line_id':[line_id],
         'year_avg':[year_avg]
    }
    )

In [ ]:
res

In [ ]:
ls.head()

In [10]:
for i,station in enumerate(station_arr):

    one_all = all[all['station_id']==station]
    one_all['entry_day'] = one_all['entry_list'].apply(lambda x: np.sum(np.array(eval(x)),axis=0))
    ls.loc[i,'year_avg'] = str(list(np.mean(one_all['entry_day'].values,axis=0)))

/usr/local/python-3.5.6/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


ZeroDivisionError: division by zero

In [ ]:
res.head()

In [ ]:
# from joblib import Parallel, delayed

# Parallel(n_jobs=-1)(delayed(
#         concatData
#         )(station,i) \
#                 for i,station in enumerate(station_arr))

In [9]:
ls

,station_id,line_id,year_avg
0,0103,01,"[244, 213, 470, 352, 551, 1120, 621, 467, 457,..."
1,0104,01,"[243, 182, 134, 203, 480, 854, 426, 349, 337, ..."
2,0105,01,"[363, 108, 179, 54, 207, 759, 330, 317, 316, 3..."
3,0106,01,"[386, 222, 69, 167, 78, 521, 374, 330, 345, 37..."
4,0107,01,"[513, 499, 234, 89, 174, 444, 372, 309, 441, 3..."
5,0108,01,"[1115, 787, 706, 460, 487, 230, 292, 257, 447,..."
6,0109,01,"[693, 463, 366, 337, 396, 337, 95, 60, 191, 18..."
7,0110,01,"[429, 310, 266, 209, 234, 206, 50, 0, 79, 78, ..."
8,0111,01,"[432, 313, 298, 281, 399, 427, 175, 83, 103, 7..."
9,0112,01,"[423, 264, 301, 320, 345, 358, 171, 85, 74, 15..."


In [ ]:
all.head()

In [ ]:
ls.dropna()


In [ ]:
ls.to_csv('ls',index=None)

In [ ]:
len(ls)

In [ ]:
his = pd.merge(ls,phy_log_rela,how='left',on='station_id')

In [ ]:
station_n = pd.read_csv('../../project_abnormal_detect/station_info.csv')
station_n['station_id'] = station_n['station_id'].apply(lambda x: '%04d'%int(x))
his = pd.merge(his,station_n,how='left',on='station_id')

In [ ]:
res = pd.merge(ls,his,on='station_id')

In [ ]:
res.head()

In [ ]:
res.drop(columns=['year_avg_y','line_id_x','line_id_y'],inplace=True)

In [ ]:
res.loc[0,'year_avg_x']

In [ ]:
def second(a):
    return a[1]


def applyMethod(lista):
    lista = eval(lista)
    ttt = []
    for i,num in enumerate(lista):
        ttt.append((i,num))
    ttt.sort(reverse=True,key=lambda x:x[1])
    indexes = [x[0] for x in ttt][:20]
    return str(list(station_arr[indexes]))

In [ ]:
res['top_20_stations'] = res['year_avg_x'].apply(applyMethod)

In [ ]:
res.head()

In [ ]:
ttttt = []
for i in range(len(res)):
    tmp = res.loc[i,'station_id']
    if tmp not in ttttt:
        ttttt.append(tmp)
    else:
        res.drop(index=i,inplace=True)
len(res)

In [ ]:
res.reset_index(inplace=True)

In [ ]:
len(res['station_id'].drop_duplicates())

In [ ]:
res

In [ ]:
top_20_list = []

stations = eval(res.loc[0,'top_20_stations'])
for s in stations:
    if s not in top_20_list:
        top_20_list.append(s)


In [ ]:
res.loc[i,'top_20_stations']

In [ ]:
len(set(top_20_list))

In [ ]:
all_huans = list(phy_log_rela['station_id'].drop_duplicates().values)

In [ ]:
phy_log_rela

In [ ]:
phy_log_rela['line_id'] = phy_log_rela['station_id'].apply(lambda x:x[:2] if len(x)==4 else x[:4])

In [ ]:
def genStationPark(x):
    lista = eval(x)
    excluede = lista + huans
    
    

In [ ]:
phy_log_rela[phy_log_rela['line_id']== '10'] 

In [ ]:
tmp = station_n.merge(phy_log_rela,on='station_id',how='left')
tmp[tmp['line_id'] == 1]

# 生成站区编码

In [ ]:
lines = list(ls['line_id'].drop_duplicates().values)
lines

In [ ]:
ls

In [ ]:
line = '94'
huans = (phy_log_rela[phy_log_rela['line_id'] == line]['station_id'].values)
huans

In [ ]:
# for i in range(len(res)):
k = 0
exclude = list(set(all_huans + eval(res.loc[k,'top_20_stations'])))
tmp_mark = ['']*len(station_arr)
for line in lines:
    indexes = list(ls[ls['line_id'] == line].index)
    huans = (phy_log_rela[phy_log_rela['line_id'] == line]['station_id'].values)
    max_index = len(huans) -1
    for num,i in enumerate(indexes):
        station = station_arr[i]
        if station in exclude:
            tmp_mark[i] = station
        else:
            if len(huans) == 1:
                tmp_mark[i] = huans[0]
            else:
                for j in range(len(huans)-1):
                    if station > huans[j] and station < huans[j+1]:
                        tmp_mark[i] = huans[j] + '*' + huans[j+1]
                    elif station < huans[j]:
                        tmp_mark[i] = '*' + huans[j]
                    elif station > huans[j+1]:
                        tmp_mark[i] = huans[j+1] +'*'
                    else:
                        tmp_mark[i] = huans[j]
    # res.loc[i,'mark'] = str(tmp_mark)

In [ ]:
# for i in range(len(res)):
k = 0
exclude = list(set(all_huans + eval(res.loc[k,'top_20_stations'])))
tmp_mark = ['']*len(station_arr)
lines = ['94']
line = '94'
indexes = list(ls[ls['line_id'] == line].index)
huans = (phy_log_rela[phy_log_rela['line_id'] == line]['station_id'].values)
max_index = len(huans) -1
for num,i in enumerate(indexes):
    station = station_arr[i]
    if station in exclude:
        tmp_mark[i] = station
    else:
#             import pdb;pdb.set_trace()
        for j in range(len(huans)-1):
            if station > huans[j] and station < huans[j+1]:
                tmp_mark[i] = huans[j] + '*' + huans[j+1]
            elif station < huans[j]:
                tmp_mark[i] = '*' + huans[j]
            elif station > huans[j+1]:
                tmp_mark[i] = huans[j+1] +'*'
# res.loc[i,'mark'] = str(tmp_mark)

In [ ]:
tmp_mark
# tmp_index = []
# for ss in range(len(tmp_mark)):
#     if tmp_mark[ss] == '':
#         tmp_index.append(ss)
# print(station_arr[tmp_index])
# print(tmp_index)

In [ ]:
huans = (phy_log_rela[phy_log_rela['line_id'] == '14']['station_id'].values)
huans

In [ ]:
num = 0;i = 358
station = station_arr[i]
station

In [ ]:
j = 0


In [ ]:
res['sation_id'] = '1449'

In [ ]:
station_arr[tmp]

In [ ]:
station_arr

In [ ]:
[i for i in huans if i[:2]=='10']

In [ ]:
pd.set_option('display.max_rows', None)  # 行
ls

In [ ]:
(phy_log_rela[phy_log_rela['line_id'] == '01']['station_id'].values)

In [ ]:
list(res[res['line_id'] == '98'].index)

In [ ]:
res.tail()

In [ ]:
res[(res['line_id'] != '83') & (res['line_id'] != '84') & (res['line_id'] != '85') & (res['line_id'] != '93') & (res['line_id'] != '04')]

In [ ]:
station_arr

In [ ]:
res[res['station_id'] == '0115']

In [ ]:
res.drop(index=[12],inplace=True)

In [ ]:
res.reset_index(drop=True)

In [ ]:
res[res['line_id']=='04']

In [ ]:
his[his['line_id_x'] == '85']

In [ ]:
ls[ls['line_id'] == '04']

In [ ]:
res.to_csv('res',index=None)

In [ ]:
ls.to_csv('ls',index=None)

In [ ]:
ls['line_id'].drop_duplicates()

In [ ]:
ls[ls['line_id']=='84']